## <center>Tensorflow 实战 RNN</center>

### 1 单步RNN: RNNCell

RNNCell 它是 Tensorflow 中实现 RNN 的基本单元,每个 RNNCell 都有一个call 方法,使用方式:
```python
(output,next_state) = call(input,state)
```
借助图片来说可能更容易理解。假设我们有一个初始状态 $h_0$，还有输入 $x_1$，调用 call($x_1$, $h_0$)后就可以得到($output_1$, $h_1$)：
<br/><br/>
<center><img src="./img/9/1.jpg" width="600"/></center>
<br/><br/>
再调用一次 call($x_1$,$h_1$) 就可以得到 ($output_2,h_2$)
<br/><br/>
<center><img src="./img/9/2.jpg" width="600"/></center>

也就是说，每调用一次 RNNCell 的 call 方法，就相当于在时间上“推进了一步”，这就是 RNNCell 的基本功能。除了call方法外，对于RNNCell，还有两个类属性比较重要：
+ state_size
+ output_size

前者是隐层的大小，后者是输出的大小。比如我们通常是将一个 batch 送入模型计算，设输入数据的形状为(batch_size, input_size)，那么计算时得到的隐层状态就是(batch_size, state_size)，输出就是(batch_size, output_size)。

In [5]:
import tensorflow as tf
import numpy as np

cell = tf.contrib.rnn.BasicRNNCell(num_units=128) # state_size = 128
print(cell.state_size) # 128

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态，形状为(batch_size, state_size)
output1, h1 = cell.__call__(inputs, h0) #调用call函数

print(h1.shape) # (32, 128)

128
(32, 128)


对于BasicLSTMCell，情况有些许不同，<font color='red'>因为LSTM可以看做有两个隐状态h和c</font>，对应的隐层就是一个Tuple，每个都是(batch_size, state_size)的形状：

In [6]:
import tensorflow as tf
import numpy as np
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = lstm_cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = lstm_cell.__call__(inputs, h0)

print(h1.h)  # shape=(32, 128)
print(h1.c)  # shape=(32, 128)

Tensor("basic_lstm_cell/Mul_2:0", shape=(32, 128), dtype=float32)
Tensor("basic_lstm_cell/Add_1:0", shape=(32, 128), dtype=float32)


### 2 多步RNN: tf.nn.dynamic_rnn

基础的 RNNCell 有一个很明显的问题：对于单个的 RNNCell，我们使用它的 call 函数进行运算时，只是在序列时间上前进了一步。比如使用 $x_1$、$h_0$ 得到 $h_1$，通过 $x_2$、$h_1$ 得到 $h_2$ 等。如果我们的序列长度为 10，就要调用 10 次 call 函数，比较麻烦。对此，TensorFlow 提供了一个 $tf.nn.dynamic\_rnn$ 函数，使用该函数就相当于调用了 $n$ 次 call 函数。即通过 {$h_0,x_1, x_2, …., x_n$} 直接得 {$h_1,h_2…,h_n$}。

具体来说，设我们输入数据的格式为 (batch_size, time_steps, input_size) ，其中 time_steps 表示序列本身的长度，如在 Char RNN 中，长度为 10 的句子对应的 time_steps 就等于10。最后的 input_size 就表示输入数据单个序列单个时间维度上固有的长度。另外我们已经定义好了一个 RNNCell，调用该 RNNCell 的 call 函数 time_steps 次

In [7]:
import tensorflow as tf
tf.reset_default_graph()
batch_size = 4 
# 通常输入数据的格式为 [timesteps,batch_size,input_data],其中input_data 表示当前RNN的输入. 
input = tf.random_normal(shape=[3, batch_size, 6], dtype=tf.float32)
# 这里使用最基本的 RNNCell ,10 表示一个RNNCell中有多少个隐藏单元
cell = tf.nn.rnn_cell.BasicRNNCell(10)
# 第一个RNNCell的输入为,输入数据和一个初始化的隐含状态,这个隐含状态通常初始化为0
#　初始化状态的大小就是 batch_size, 表明有 batch_size 个独立的状态
init_state = cell.zero_state(batch_size, dtype=tf.float32)
# output 表示每一个RNNCell 的输出,输出的形状为(3,4,10)
# final_state 表示最后一个 RNNCell 的输出状态,输出形状为(4,10)
output, final_state = tf.nn.dynamic_rnn(cell, input, initial_state=init_state, time_major=True)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(output).shape)
    print(sess.run(final_state))
    print(sess.run(final_state).shape)

(3, 4, 10)
[[ 0.15897425 -0.49791458 -0.4963269   0.5887911   0.8083712   0.20863965
  -0.02076925 -0.7549604   0.2806686   0.36615604]
 [ 0.78095293  0.938       0.60008585  0.5948752   0.25974703  0.039651
  -0.29459378  0.32053632  0.44302717  0.5970255 ]
 [-0.6674305   0.98681504  0.96141636 -0.6867633   0.50147516  0.31646675
  -0.64991957  0.92567086  0.43790862  0.65995336]
 [ 0.19428924 -0.7440126  -0.6446582  -0.8749729  -0.21022728 -0.7555016
   0.15203257  0.8128691  -0.9370999  -0.7597995 ]]
(4, 10)


### 3 堆叠RNNCell: MultiRNNCell

很多时候，单层 RNN 的能力有限，我们需要多层的 RNN。将 $x$ 输入第一层 RNN 的后得到隐层状态 $h$,这个隐层状态就相当于第二层 RNN 的输入，第二层 RNN 的隐层状态又相当于第三层 RNN 的输入，以此类推。在TensorFlow 中，可以使用 $tf.nn.rnn\_cell.MultiRNNCell$ 函数对 RNNCell 进行堆叠，相应的示例程序如下：

In [11]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
# 每调用一次这个函数就返回一个BasicRNNCell
def get_a_cell():
    return tf.nn.rnn_cell.BasicRNNCell(num_units=128)
# 用tf.nn.rnn_cell MultiRNNCell创建3层RNN
cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(3)]) # 3层RNN
# 得到的cell实际也是RNNCell的子类
# 它的state_size是(128, 128, 128)
# (128, 128, 128)并不是128x128x128的意思
# 而是表示共有3个隐层状态，每个隐层状态的大小为128
print(cell.state_size) # (128, 128, 128)
# 使用对应的call函数
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = cell.call(inputs, h0)
print(h1) # tuple中含有3个32x128的向量

(128, 128, 128)
(<tf.Tensor 'cell_0/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_1/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_2/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>)
